In [1]:
#! pip install bmipy
#! pip install eto
import sys 
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import bmi_cfe_perturb_ens
import bmi_pf
import bmi_cfe
import bmi_PET
from fao_pet import ETRCalculator

In [2]:
pet_instance = bmi_PET.BMI_pet_model()
pet_instance.initialize(cfg_file='./pet.json', current_time_step=1)
pet_instance.update()
pet_instance.finalize()
pet = pet_instance.get_value('etr').values

In [3]:
enkf_instance = bmi_pf.particle_filter_bmi('./pf.json')
cfe_purturb_instance = bmi_cfe_perturb_ens.BMI_CFE('./new_cfe_perturb.json')
cfe_instance = bmi_cfe.BMI_CFE('./new_cfe.json')

AttributeError: 'particle_filter_bmi' object has no attribute 'simulations'

In [ ]:
#initialization
cfe_purturb_instance.initialize()
cfe_instance.initialize()
enkf_instance.initialize()

In [ ]:
cfe_purturb_inputs = cfe_purturb_instance.get_input_var_names()
input_lists_cfe_purturb = {inputs:{ens:[] for ens in range(0)} for inputs in cfe_purturb_inputs}
print("CFE Perturbed code inputs are:", input_lists_cfe_purturb)

cfe_inputs = cfe_instance.get_input_var_names()
input_lists_cfe = {inputs:{ens:[] for ens in range(0)} for inputs in cfe_inputs}
print("CFE code inputs are:", input_lists_cfe)

enkf_inputs = enkf_instance.get_input_var_names()
input_lists_enkf = {inputs:{ens:[] for ens in range(0)} for inputs in enkf_inputs}
print("EnKF code inputs are:", input_lists_enkf)


cfe_purturb_outputs = cfe_purturb_instance.get_output_var_names()
output_lists_cfe_purturb = {output:{ens:[] for ens in range(0)} for output in cfe_purturb_outputs}
print("CFE Perturbed code outputs are:", output_lists_cfe_purturb)


cfe_outputs = cfe_instance.get_output_var_names()
output_lists_cfe = {output:{ens:[] for ens in range(0)} for output in cfe_outputs}
print("CFE code outputs are:", output_lists_cfe)

enkf_outputs = enkf_instance.get_output_var_names()
output_lists_enkf = {output:{ens:[] for ens in range(0)} for output in enkf_outputs}
print("EnKF code outputs are:", output_lists_enkf)

In [ ]:
cfe_purturb_outputs = cfe_purturb_instance.get_output_var_names()
output_lists_cfe_purturb = {output:{ens:[] for ens in range(cfe_purturb_instance.n_cfe_ensembles)} for output in cfe_outputs}

cfe_outputs = cfe_instance.get_output_var_names()
output_lists_cfe = {output:{ens:[] for ens in range(cfe_purturb_instance.n_cfe_ensembles)} for output in cfe_outputs}

enkf_outputs = enkf_instance.get_output_var_names()
output_lists_enkf = {output:{ens:[] for ens in range(cfe_purturb_instance.n_cfe_ensembles)} for output in enkf_outputs}

In [ ]:
### Observation data

observation_no_noise = []


with open(cfe_instance.forcing_file, 'r') as f:
    df_forcing = pd.read_csv(f)
    
for t, precip in enumerate(df_forcing['APCP_surface']):
    cfe_instance.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
    cfe_instance.set_value('water_potential_evaporation_flux', pet[t])
    cfe_instance.update()
    obs = cfe_instance.get_value("land_surface_water__runoff_volume_flux")
    observation_no_noise.append(obs)
    
cfe_instance.finalize()

perturbation_factor_obs = np.random.standard_normal(size=None)
error_factor_obs = 0.15
observation_no_noise = np.array(observation_no_noise)       
observation_noisy = (perturbation_factor_obs * observation_no_noise * error_factor_obs) + observation_no_noise




In [ ]:
######Synthetic loop

cfe_purturb_instance.initialize()
cfe_instance.initialize()


with open(cfe_purturb_instance.forcing_file, 'r') as f:
    df_forcing = pd.read_csv(f)


da_simulations_all = []
da_simulations_all_mean = []
state_variables_all = []
state_variables_all_mean = []
simulations_all = []
observations_all = []
updated_state_all = []
updated_simulations_all = []
gw_state_all = []
gw_state_all_mean = []
updated_gw_state_all = []


for t, precip in enumerate(df_forcing['APCP_surface']):
    
    cfe_purturb_instance.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
    cfe_purturb_instance.set_value('water_potential_evaporation_flux', pet[t])
    cfe_purturb_instance.update()
    simulations = cfe_purturb_instance.get_value("land_surface_water__runoff_volume_flux")
    simulations = np.array(list(simulations.values()))
    da_simulations_all.append(simulations)
    mean_simulations = np.mean(simulations)
    da_simulations_all_mean.append(mean_simulations)
    
    state_variables = cfe_purturb_instance.get_value("SOIL_CONCEPTUAL_STORAGE")
    state_variablesssss = np.array(list(state_variables))
    state_variables_all.append(state_variables)
    state_variables_mean = np.mean(state_variablesssss)
    state_variables_all_mean.append(state_variables_mean)
    
    gw_state = cfe_purturb_instance.get_value("DEEP_GW_TO_CHANNEL_FLUX")
    gw_statesssss = np.array(list(gw_state))
    gw_state_all.append(gw_state)
    gw_state_mean = np.mean(gw_statesssss)
    gw_state_all_mean.append(gw_state_mean)
    

    Flow = observation_noisy[t]
    observations_all.append(Flow)
    
    enkf_instance.set_value('Observations', Flow) 
    enkf_instance.set_value('Simulations', simulations)
    enkf_instance.set_value('State Variables', state_variables)
    enkf_instance.update()
    updated_state_sm = enkf_instance.get_value("state_estimates")
    updated_state_all.append(updated_state_sm)
    updated_state_sm = updated_state_sm[0]
    
    enkf_instance.finalize()
    cfe_instance.initialize()
    print(simulations)

    enkf_instance.set_value('Observations', Flow) 
    enkf_instance.set_value('Simulations', simulations)
    enkf_instance.set_value('State Variables', gw_state)
    enkf_instance.update()
    updated_state_gw = enkf_instance.get_value("state_estimates")
    updated_gw_state_all.append(updated_state_gw)
    updated_state_gw = updated_state_gw[0]
    
    enkf_instance.finalize()
    cfe_instance.initialize()

    cfe_instance.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
    cfe_instance.set_value('water_potential_evaporation_flux', pet[t])
    cfe_instance.set_value('SOIL_CONCEPTUAL_STORAGE', updated_state_sm)
    cfe_instance.set_value('DEEP_GW_TO_CHANNEL_FLUX', updated_state_gw)
    cfe_instance.update()
    simulation_updated = cfe_instance.get_value("land_surface_water__runoff_volume_flux")
    updated_simulations_all.append(simulation_updated)
    
    list_of_states_sm = [updated_state_sm] * 100
    cfe_purturb_instance.set_value('SOIL_CONCEPTUAL_STORAGE', list_of_states_sm)
    
    
    list_of_states_gw = [updated_state_gw] * 100
    cfe_purturb_instance.set_value('DEEP_GW_TO_CHANNEL_FLUX', list_of_states_gw)
    
cfe_instance.finalize()
cfe_purturb_instance.finalize()
enkf_instance.finalize()

